In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from CinePred.new_model import *
from CinePred.data.preprocessing import *
from CinePred.data.importing import import_data
from CinePred.data.featuring import add_inflation

In [10]:
df = import_data('../raw_data/IMDb_movies.csv')

In [11]:
df.head()

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0


In [5]:
df = keep_columns(df,
                      column_names=[
                          'year', 'date_published', 'genre', 'duration',
                          'budget', 'worlwide_gross_income'
                      ])
df = remove_na_rows(df)

In [6]:
# date_published
df['date_published'] = convert_to_date(df[['date_published']])

# day of the year
df['date_sin'] = add_sin_features(df[['date_published']])
df['date_cos'] = add_cos_features(df[['date_published']])
df.drop(columns='date_published', inplace=True)

# year, duration
df['year'] = convert_to_int(df[['year']])
df['duration'] = convert_to_int(df[['duration']])

# genre
ohe = GenreOHE()
ohe.fit(df) # la colonne 'genre' est spécifié dans la classe
df = ohe.transform(df)

# budget
df['budget'] = convert_budget_column(df[['budget']])
df = df[df['budget'] != 0]
df = add_inflation(df, 'budget')
df['budget'] = log_transformation(df[['budget']])

In [7]:
pd.set_option('display.max_rows', None)

In [8]:
df['worlwide_gross_income'] = convert_income(df[['worlwide_gross_income']])

df = add_inflation(df, 'worlwide_gross_income')

df['worlwide_gross_income'] = log_transformation(df[['worlwide_gross_income']])

TypeError: Years can only be converted to other years. Months only to other months.

In [9]:
df.head()

,year,duration,budget,worlwide_gross_income,date_sin,date_cos,Action,Biography,Thriller,Crime,...,Sport,History,War,Horror,Family,Music,Sci-Fi,Animation,Comedy,year_2
0,1920,76,5.367225,8811,8.660254e-01,0.500000,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1920
1,1921,107,5.605531,4272,-2.449294e-16,1.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1921
2,1921,150,7.063220,9183673,8.660254e-01,-0.500000,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1921
3,1921,68,6.558070,26916,-5.000000e-01,0.866025,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1921
4,1923,82,6.725294,11233,1.224647e-16,-1.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1923
